In [1]:
options nosource nonotes;
/* =============
    1. BLOG POST - Please note I have chosen to use GitHub / Jupyter notebooks in substitute 
        of Tumblr for my blog assignments
    ============

    *Please note that instead of signing up for a Tumblr account, I have chosen to leverage 
        an already existing account on Github to consolidate work and platforms.

    For this exercise I have binned the majority of variables for better categorization 
        and interpretation. In addition, deriving the defaulted variable from existing 
        variables to determine if a loan was defaulted.

    Variables: 
        1) purpose - This is the purpose of why the loan was requested.  
            A high level grouping of the loan types.
            
        2) int_rate - What interest rate the loan is at
            binned with variable "int_rate_bin"
            
        3) dti - Debt to income ratio
            binned with variable "dti_bin"
            
        4) revol_util - revolving credit utilization
            binned with variable "revol_util_bin"
            
        5) defaulted - Dervied variable from the columns "recoveries" and "collection_recovery_fee".  
            If either of these have values populated greater than 0, then we can assume 
            that a default event occured.  
*/

In [2]:
/* This option might pop an error, but reduces all the log lines of code when loading the data set 
- to shorten the notebook.  You can ignore the error "Expecting page 1 ..." */
options nosource nonotes;

/* Impor the file */
proc import datafile = '/folders/myfolders/sasuser.v94/LoanStats3a.csv'
out = work.loanstats
dbms = CSV;
run;

In [3]:
/* Set a new data set */
DATA new; set work.loanstats;

In [4]:
/* Here I need to convert some data */ /* int_rate = 25% - needs to be adjusted to .25 */
/* might be an easier way - but new to SAS syntax, and want to work at data scrubbing */

/* set new variable */
int_rate_d = int_rate;

/* remove '%' */
int_rate_d = substr(int_rate_d,1,index(int_rate_d, '%')-1);

/* convert to decimal */
int_rate_d = input(int_rate_d, 8.);

/* divide by 100 to get decimal notation */
int_rate_d = int_rate_d / 100;


/* same with revoling utilization */
revol_util_d = revol_util;
revol_util_d = substr(revol_util_d,1,index(revol_util_d, '%')-1);
revol_util_d = input(revol_util_d, 8.);
revol_util_d = revol_util_d / 100;

In [5]:
/* Variable Adding / Binning */

/* If the recoveries or collection_recovery_fee > 0 then assign a default value of 1 */
IF recoveries > 0 THEN defaulted = 1;
ELSE IF collection_recovery_fee > 0 THEN defaulted = 1;
ELSE defaulted = 0;

/* int_rate */
length int_rate_bin $20; /* Set length */
IF int_rate_d < 0.10 THEN int_rate_bin = "< 10%";
ELSE IF int_rate_d >= 0.10 AND int_rate_d < 0.15 THEN int_rate_bin = "10% - 15%";
ELSE IF int_rate_d >= 0.15 AND int_rate_d < 0.20 THEN int_rate_bin = "15% - 20%";
ELSE IF int_rate_d >= 0.20 AND int_rate_d < 0.25 THEN int_rate_bin = "20% - 25%";
ELSE IF int_rate_d >= 0.25 AND int_rate_d < 0.30 THEN int_rate_bin = "25% - 30%";
ELSE IF int_rate_d >= 0.30 THEN int_rate_bin = "> 30%";
ELSE int_rate_bin = "N/A"; /* Catch bad data */

/* dti - range of 0-30 */
length dti_bin $20; /* Set length */
IF dti < 1 THEN dti_bin = "< 1";
ELSE IF dti >= 1 AND dti < 10 THEN dti_bin = "1 - 10";
ELSE IF dti >= 10 AND dti < 20 THEN dti_bin = "10 - 20";
ELSE IF dti >= 20 AND dti < 30 THEN dti_bin = "20 - 30";
ELSE IF dti >= 30 THEN dti_bin = "> 30";
ELSE dti_bin = "N/A"; 

/* revol_util */
length revol_util_bin $20; /* Set length */
IF revol_util_d < 0.25 THEN revol_util_bin = "< 25%";
ELSE IF revol_util_d >= 0.25 AND revol_util_d < 0.50 THEN revol_util_bin = "25% - 50%";
ELSE IF revol_util_d >= 0.50 AND revol_util_d < 0.75 THEN revol_util_bin = "50% - 75%";
ELSE IF revol_util_d >= 0.75 AND revol_util_d < 1 THEN revol_util_bin = "75% - 100%";
ELSE IF revol_util_d >= 1 THEN revol_util_bin = "> 100%";
ELSE revol_util_bin = "N/A"; 

/* Set some labels */
LABEL   purpose = "Purpose for Loan"
        int_rate = "interest rate"
        dti = "debt to income ratio"
        revol_util = "revolving utilization"

        int_rate_bin = "interest rate (binned)"
        dti_bin = "debt to income ratio (binned)"
        revol_util_bin = "revolving utilization (binned)"
        defaulted = "Defulted Loan";

In [6]:
/* purpose", int_rate", "dti", "revol_util", "recoveries" and defaulted  */

/* Run some distribution frequencies */
Proc freq; tables purpose int_rate_bin dti_bin revol_util_bin defaulted;
run;